In [2]:
from numpy import info
import requests
from requests import Request, Session
import numpy as np
import io
import pandas as pd
from datetime import datetime
import json
from aioconnect.helpers import *
from dotenv import load_dotenv
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [3]:
import aioconnect

In [4]:
def get_token(
    email: str,
    password: str,
):
    """Log into AIO Impact and get a token.

    Parameters
    ----------
    email : str
        Email address of the user account.

    password : str
        Password of the user account.

    Returns
    -------
    str
        Bearer token.

    Raises
    -------
    requests.exceptions.HTTPError
        If the username and password are wrong.

    Examples
    --------
    >>> aioconnect.get_token(
    >>>     email="firstname.lastname@aioneers.com", password="xxx",
    >>> )
    """

    url = getenv("CONNECT_URL") + "/login"
    payload = json.dumps({"email": email, "password": password})
    headers = {"Content-Type": "application/json"}

    response = requests.request("POST", url, headers=headers, data=payload)
    response.raise_for_status()

    token = response.json()["token"]

    return token

In [5]:
def test_correct_credentials():
    password = aioconnect.vault_get_secret(
        scope="aio-data-science-key", key="sebastian-szilvas-aio-impact"
    )

    res = get_token(
        email="sebastian.szilvas@aioneers.com",
        password=f"{password}",
    )

    assert isinstance(res, str)
    assert len(res) > 250

def test_wrong_password():
    try:
        res = get_token(
            email="sebastian.szilvas@aioneers.com",
            password="wrong password",
        )
    except requests.exceptions.HTTPError as exception:
        assert exception.response.status_code == 401


In [5]:
test_correct_credentials()
test_wrong_password()

In [10]:
def convert_to_json(df):
    """
    The function will convert the pandas dataframe to json file.
    -----------------------------

    Parameters : 
    -----------------------------
    df -> Pandas.DataFrame : It is the dataframe which you want to convert into json format.

    Return : 
    -----------------------------
    output -> json : It is the data converted into json format. 

    """
    # Checking Conditions

    # 1) Unique Columns
    if df['externalId'].isna().any() or df['name'].isna().any() or df['actuals value'].isna().any() or df['actuals timestamp'].isna().any():
        raise ValueError('Column Names : externalId, name, actuals value and actuals timestamp cannot be kept empty.')
    
    # Similar ExternalIds should not have same actuals timestamp
    df_2 = df.copy()
    for i in df_2['externalId'].unique():
        temp = df_2[df_2['externalId'] == i]
        if temp['actuals timestamp'].nunique() != len(temp['actuals timestamp']):
            raise ValueError('There are more than 1 similar actuals timestamp for externalId {0}'.format(i))


    # 2) Optional Columns    
    if 'projections value' in df.columns and 'projections timestamp' not in df.columns:
        raise KeyError('projection timestamp missing for a projection value')
    elif 'projections value' not in df.columns and 'projections timestamp' in df.columns:
        raise KeyError('projection value missing for a projection timestamp')
    elif 'projections value' not in df.columns and 'projections timestamp' not in df.columns:
        pass
    else:
        df['projections value'].replace({"-": 0}, inplace=True)
        if df['projections timestamp'].nunique() != len(df['projections timestamp']):
            raise ValueError('There are more than 1 similar actuals timestamp')

    if 'target' in df.columns and 'targetDate' not in df.columns:
        raise KeyError('Target date missing for a target value')
    elif 'target' not in df.columns and 'targetDate' in df.columns:
        raise KeyError('Target value missing for a target date')
    elif 'target' not in df.columns and 'targetDate' not in df.columns:
        pass
    else:
        df['target'].replace({"-": 0}, inplace=True)

    # df_2 = df.drop(['externalId', 'name', 'description', 'actuals value', 'actuals timestamp', 'target value',
    #     'target timestamp', 'projections value', 'projections timestamp'], axis=1)
    if 'measureId' in df.columns:
        if 'initiativeId' in df.columns:

            df_2 = df[['initiativeId','measureId']]
            m_notnull = pd.notnull(df_2["measureId"])

            df_3 = df_2[m_notnull]

            if df_3['initiativeId'].isna().any():
                raise ValueError('If measureId is provided then initiativeId is mandatory.')
        else:
            raise ValueError('If measureId is provided then initiativeId is mandatory.')

    if 'metricId' in df.columns:    
        if 'initiativeId' in df.columns:
            
            df_4 = df[['initiativeId','metricId']]

            i_null = pd.isnull(df_4["initiativeId"])
        
            df_5 = df_4[i_null]

            if df_5['metricId'].isna().any():
                raise ValueError('If initiativeId is not provided then metricId should be provided.')
        else: 
            if df['metricId'].isna().any():
                raise ValueError('If initiativeId is not provided then metricId should be provided.')
    else:
        if 'initiativeId' not in df.columns:
            raise ValueError('If initiativeId is not provided then metricId should be provided.')
        
    # Grouping columns
    finalList = []
    df['actuals value'].replace({"-": 0}, inplace=True)

    # if 'target' in df.columns:
    #     df['target'].replace({"-": 0}, inplace=True)

    
    grouped = df.groupby(['externalId','actuals timestamp'])

    for key, value in grouped:
        
        dictionary = {}

        j = grouped.get_group(key).reset_index(drop=True)
        dictionary['externalId'] = j.at[0, 'externalId']
        dictionary['name'] = j.at[0, 'name']
        dictionary['description'] = j.at[0, 'description']

        if 'initiativeId' in df.columns:
            if not(pd.isna(j.at[0, 'initiativeId'])):
                dictionary['initiativeId'] = j.at[0, 'initiativeId']

        if 'measureId' in df.columns:
            if not(pd.isna(j.at[0, 'measureId'])):
                dictionary['measureId'] = j.at[0, 'measureId']

        if 'metricId' in df.columns:
            if not(pd.isna(j.at[0, 'metricId'])):
                dictionary['metricId'] = j.at[0, 'metricId']
        
        if 'target' in df.columns:
            if not(pd.isna(j.at[0, 'target'])):
                dictionary['target'] = j.at[0, 'target']
                dictionary['targetDate'] = j.at[0, 'targetDate']
        
        dict_actual = []
        dict_projection = []
        
        anotherDict_actual = {}
        anotherDict_projections = {}
        
        for i in j.index:

            anotherDict_actual['value'] = j.at[i, 'actuals value']
            anotherDict_actual['timestamp'] = j.at[i, 'actuals timestamp']

            if 'projections value' in df.columns:
                # print(j.at[i, 'projections value'])
                # print(not(pd.isna(j.at[i, 'projections value'])))
                if not(pd.isna(j.at[i, 'projections value'])):
                    anotherDict_projections['value'] = j.at[i, 'projections value']
                    anotherDict_projections['timestamp'] = j.at[i, 'projections timestamp']

                    dictionary_copy_projection = anotherDict_projections.copy()
                    dict_projection.append(dictionary_copy_projection)

                    dictionary['projections'] = dict_projection
            
            dictionary_copy_actual = anotherDict_actual.copy()
            dict_actual.append(dictionary_copy_actual)

            # dictionary_copy_projection = anotherDict_projections.copy()
            # dict_projection.append(dictionary_copy_projection)
            
        dictionary['actuals'] = dict_actual
        # dictionary['projections'] = dict_projection
        
        dict_copy = dictionary.copy()
        finalList.append(dict_copy)

    # Dump the data into output variable
    output = json.dumps(finalList, sort_keys = True, indent=4, cls=NumpyEncoder)
    # output = json.dumps(finalList, sort_keys = True, indent=4, cls=NumpyEncoder).replace("NaN" , 'null')
    
    return output

class NumpyEncoder(json.JSONEncoder):
    """ Custom encoder for numpy data types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):

            return int(obj)

        elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
            return float(obj)

        elif isinstance(obj, (np.complex_, np.complex64, np.complex128)):
            return {'real': obj.real, 'imag': obj.imag}

        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()

        elif isinstance(obj, (np.bool_)):
            return bool(obj)

        elif isinstance(obj, (np.void)): 
            return None

        return json.JSONEncoder.default(self, obj)

In [7]:
def upsert_DOT(
    token: str,
    dataframe: pd.DataFrame,
    limit: int = 100
) -> list:
    """Create a new DOT in AIO Impact or update it if the DOT is already existing.

    Parameters
    ----------
    token : str
        Token which was returned from the user login.

    dataframe : Pandas.DataFrame
        Dataframe containing DOT details.
    
    limit : int
        integer value that indicates minimum value of DOTs to be sent. By default, the limit is set to 100

    Returns
    -------
    requests.Response
        HTTP response.

    Examples
    --------
    >>> token = aioconnect.get_token(
    >>> email="firstname.lastname@aioneers.com", password="xxx",
    >>> )
    >>> res = aioconnect.upsert_DOT(
    >>>     token=token,
    >>>     dataframe = df
    >>> )
    """

    url = getenv("CONNECT_URL") + "/dots"

    columns_list = [
        "externalId",
        "name",
        "description",
        "metricId",
        'initiativeId',
        'measureId',
        "actuals value",
        "actuals timestamp",
        "target",
        "targetDate",
        "projections value",
        "projections timestamp"
    ]

    for i in dataframe.columns:
        if i not in columns_list:
            raise KeyError("Columns not correct")

    if len(dataframe)<limit:
        limit = len(dataframe)
    
    for i in range(0, len(dataframe), limit):
        slc = dataframe.iloc[i : i + limit]
        payload = convert_to_json(slc)

        for attempt in range(20):
            try:
                response = requests.put(
                url=url, headers={"Authorization": f"Bearer {token}",
                                    'Content-Type': 'application/json'}, data=payload)
                response.raise_for_status()
            except requests.exceptions.HTTPError as exception:
                print("There was a failed attempt")
                print(exception)
            else:
                break
        # print(payload)
        # response = requests.put(
        #         url=url, headers={"Authorization": f"Bearer {token}",
        #                             'Content-Type': 'application/json'}, data=payload)
        # response.raise_for_status()

    return response

In [8]:
password = aioconnect.vault_get_secret(
        scope="aio-data-science-key", key="sebastian-szilvas-aio-impact"
    )

token = get_token(
    email="sebastian.szilvas@aioneers.com",
    password=f"{password}",
)


def test_single_DOT_correct_headers():
    data = [{

        "actuals timestamp": "2022-01-05T06:28:30.000Z",

        "actuals value": "-",

        "description": "Test DOT1",

        "externalId": "201",

        "initiativeId": np.NaN,

        "measureId": np.NaN,

        "metricId": 101,

        "name": "Test DOT1",
        
        "target": 590,
        
        "targetDate": "2021-07-30T00:00:00+0000",

        "projections value": 59,
        
        "projections timestamp": "2021-07-30T00:00:00+0000"
    },
    {

        "actuals timestamp": "2022-01-05T06:28:30.000Z",

        "actuals value": 291.2,

        "description": "Test DOT2",

        "externalId": "202",

        "initiativeId": "cd0eef61-5d45-4753-8436-ed12e113a94a",

        "measureId": "2f4fffd9-2681-4719-93da-ee6add07fb70",

        "metricId": 101,

        "name": "Test DOT2",

        "target": 690,
        
        "targetDate": "2021-07-30T00:00:00+0000",

        "projections value": 59,
        
        "projections timestamp": "2021-07-30T00:00:00+0000"
    },
    {

        "actuals timestamp": "2022-01-05T06:28:30.000Z",

        "actuals value": 281.2,

        "description": "Test DOT3",

        "externalId": "203",

        "initiativeId": "cd0eef61-5d45-4753-8436-ed12e113a94a",

        "measureId": "2f4fffd9-2681-4719-93da-ee6add07fb70",

        "metricId": 101,

        "name": "Test DOT3",

        "target": 700,
        
        "targetDate": "2021-07-30T00:00:00+0000"
    },
    {

        "actuals timestamp": "2022-01-05T06:28:30.000Z",

        "actuals value": 291.2,

        "description": "Test DOT4",

        "externalId": "204",

        "initiativeId": "ee51bd64-3a19-45b9-83ed-4044f9e0b17e",

        "measureId": "5230ceca-7528-4a72-a149-0f6499ec9db1",

        "metricId": 101,

        "name": "Test DOT4",

        "target": 800,
        
        "targetDate": "2021-07-30T00:00:00+0000"
    }
    
]

    
    
    df = pd.DataFrame.from_dict(data)

    # By default : limit is set to 100
    limit = 2
    res = upsert_DOT(
        token=token, dataframe=df, limit = limit)

    assert res.status_code == 200
    # assert (res.json()["created"] == len(df.index) or
    #         res.json()["updated"] == len(df.index))
    assert (res.json()["created"] == limit or
            res.json()["updated"] == limit)


def test_two_DOTs_correct_headers():
    data = [{

        "actuals timestamp": "2022-01-05T06:28:30.000Z",

        "actuals value": 281.2,

        "description": "Test DOT1",

        "externalId": "201",

        "initiativeId": np.NaN,

        "measureId": np.NaN,

        "metricId": 101,

        "name": "Test DOT1",
        
        "target": 590,
        
        "targetDate": "2021-07-30T00:00:00+0000",

        "projections value": 59,
        
        "projections timestamp": "2021-07-30T00:00:00+0000"
    }]
    # data = {
    #             "externalId": ["2343284","73894792"],
    #             "name": ["helloooooo","helloooooo2"],
    #             "description": ["sdklfjslfdjsldf","sdklfjslfdjsldf"],
    #             "metricId": [10,10],
    #             "actuals": [[
    #                 {
    #                         "timestamp": "2021-12-07T14:59:38.273Z",
    #                         "value": 15
    #                 }
    #             ],
    #             [
    #                 {
    #                         "timestamp": "2021-12-07T14:59:38.273Z",
    #                         "value": 17
    #                 }
    #             ]],
    #         }
    df = pd.DataFrame.from_dict(data)
    limit = 2
    res = upsert_DOT(token = token,
        dataframe= df, limit = limit
    )

    assert res.status_code == 200
    assert (res.json()["created"] == limit or
            res.json()["updated"] == limit)


def test_null_DOTs():
    data = [{

        "actuals timestamp": "2022-01-05T06:28:30.0002",

        "actuals value": 2,

        "description": "Test DOT1",

        "externalId": "TD7",

        "initiativeId": "9cc5c76f-7cc0-43e9-8f69-9549697c2955",

        "metricId": 10,

        "name": "Test DOT1"
    },
    {

        "actuals timestamp": "2022-01-06T06:28:30.0001",

        "actuals value": 3,

        "description": "Test DOT2",

        "externalId": "TD8",

        "initiativeId": "9cc5c76f-7cc0-43e9-8f69-9549697c2955",

        "metricId": 10,

        "name": "Test DOT2"
    },
    {

        "actuals timestamp": "2022-01-07T06:28:30.0000",

        "actuals value": 4,

        "description": "Test DOT3",

        "externalId": "TD9",

        "initiativeId": "9cc5c76f-7cc0-43e9-8f69-9549697c2955",

        "metricId": 10,

        "name": "Test DOT3"
    }]
    
    df = pd.DataFrame.from_dict(data)
    limit = 2
    res = upsert_DOT(token = token,
        dataframe= df, limit = limit
    )

    assert res.status_code == 200
    assert (res.json()["created"] == 1 or
            res.json()["updated"] == 1)

In [13]:
test_single_DOT_correct_headers()

In [12]:
test_two_DOTs_correct_headers()

In [11]:
test_null_DOTs()

In [24]:
test_single_DOT_correct_headers()
test_two_DOTs_correct_headers()

  externalId         name      description  metricId  \
0    2343284   helloooooo  sdklfjslfdjsldf        10   
1   73894792  helloooooo2  sdklfjslfdjsldf        10   

                                             actuals  
0  [{'timestamp': '2021-12-07T14:59:38.273Z', 'va...  
1  [{'timestamp': '2021-12-07T14:59:38.273Z', 'va...  


# This function still needs fixing

In [ ]:
# def get_object(
#     token: str,
#     object: str = "digitalObjectTwins",
# ) -> list:
#     """Get JSON object.

#     Parameters
#     ----------
#     token : str
#         Token which was returned from the user login.
#     object : str = "dotTypes"
#         Object to be extracted from the API.

#     Returns
#     -------
#     list
#         List of JSON objects.

#     Raises
#     ------
#     KeyError
#         Raises KeyError when the input is not correct.
#     """
#     url = getenv("CONNECT_URL")

#     if object == "digitalObjectTwins":
#         url += "/digitalObjectTwins/"
#     elif object == "measures":
#         url += "/measures/"
#     elif object == "initiatives":
#         url += "/initiatives/"
#     else:
#         raise KeyError

#     headers = {"Authorization": f"Bearer {token}"}

#     response = requests.request("GET", url, headers=headers)
#     response.raise_for_status()
#     return response.json()